In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
import pandas as pd
from plot import color_pick

np.random.seed(0)
# labels
n_classes = 10  
df = pd.read_csv('../result/P300_600-R300_1200.csv')
y_true = df['true_list'].to_numpy()
y_scores = df['predicted_list'].to_numpy()

# Binarization of true and predicted labels
y_true_bin = label_binarize(y_true, classes=range(n_classes))
y_scores_bin = label_binarize(y_scores, classes=range(n_classes))

# Calculate true and false positive rates for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_scores_bin[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Calculate micro-average of true and false positive rates
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_scores_bin.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Calculate macro-average of true and false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= n_classes
fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

class_list = ['ABA', 'CFR', 'ECO', 'EFA', 'KOX', 'KPN', 'PMA', 'SAU', 'SHO', 'SMA']

# plot ROC
plt.figure(figsize=(10, 8))
plt.plot(fpr["micro"], tpr["micro"], label='micro-average ROC curve (area = {0:0.2f})'.format(roc_auc["micro"]), color='deeppink', linestyle=':', linewidth=2)
plt.plot(fpr["macro"], tpr["macro"], label='macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]), color='navy', linestyle=':', linewidth=2)
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'green', 'red', 'purple', 'yellow', 'brown', 'pink', 'gray'])
for i, c in enumerate(class_list):
    plt.plot(fpr[i], tpr[i], color=color_pick(i), lw=2, label='ROC curve of {0} (AUC = {1:0.2f})'.format(c, roc_auc[i]))
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.03])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC: Multi-class')
plt.legend(loc="lower right")
plt.savefig('P300_600-R300_1200.png', bbox_inches='tight', pad_inches=0.1, dpi=300)
plt.show()